# Predykcja wytrzymałości na ściskanie na podstawie rzeczywistych danych wpisanych

Za pomocą bibloteki "pickle" model został wyciągnięty oraz zserializowany w celu dalszego użytkowania. Dużym plusem jest to że nie potrzeba instalować dodatkowych bibliotek oprócz standardowej biblioteki "pickle"


Uruchamiaj po kolei poszczególne kolumny aby móc edytować wartości w pliku Excel

In [2]:
import pickle
import pandas as pd
import xlwings as xw

W tym kroku przypiszemy wartości poszczególnych składników mieszaki betonowej

In [3]:
rows = [430,0, 0, 178.5, 8.0, 1001.9, 688.7, 28]

In [4]:
columns = ['cement [kg/m3]',"żużel_wielkopiecowy [kg/m3]",
       'popiół_lotny [kg/m3]', 'woda [kg/m3]',
       'superplastyfikator [kg/m3]', 'kruszywo_grube [kg/m3]',
       'kruszywo_drobne [kg/m3]', 'wiek [dni]']

In [5]:
predicted = pd.DataFrame([dict(zip(columns,rows))])
predicted

,cement [kg/m3],żużel_wielkopiecowy [kg/m3],popiół_lotny [kg/m3],woda [kg/m3],superplastyfikator [kg/m3],kruszywo_grube [kg/m3],kruszywo_drobne [kg/m3],wiek [dni]
0,430,0,0,178.5,8.0,1001.9,688.7,28


Wyświetlanie danych w excelu.

In [6]:
xw.view(predicted)

## Otwieramy teraz zapisane wcześniej modele

Model oparty na drzewach decyzyjnych

In [7]:
with open (r"C:\Users\Damian\Desktop\xD\Projekty\concrete-strenght-calculator\modelPickle", "rb") as model_dat:
    model = pickle.load(model_dat)

Model wykorzystujący strukturę wzmacniającą gradient

In [8]:
with open (r"C:\Users\Damian\Desktop\xD\Projekty\concrete-strenght-calculator\modelPickle2", "rb") as model_dat2:
    model2 = pickle.load(model_dat2)

I podstawiamy nasze przykładowe dane do wytrenowanego modelu

In [9]:
print(model.predict(predicted)[0], "MPa") #Drzewo decyzyjne

52.3 MPa


In [10]:
print(model2.predict(predicted)[0], "MPa") #Gradient boost

51.195663093324114 MPa


## Zedytuj teraz plik Excel podstawiając własne dane :)

Możesz teraz zedytować swoje wartości w otwartym pliku Excel. 

Po edycji wykonaj poniższą komórkę


In [11]:
predicted2 = xw.books.active.sheets[0].range("A1:I2").options(pd.DataFrame).value
predicted2


,cement [kg/m3],żużel_wielkopiecowy [kg/m3],popiół_lotny [kg/m3],woda [kg/m3],superplastyfikator [kg/m3],kruszywo_grube [kg/m3],kruszywo_drobne [kg/m3],wiek [dni]
,,,,,,,,
0.0,430.0,0.0,0.0,178.5,8.0,1001.9,688.7,28.0


In [12]:
wynik_dt = model.predict(predicted2)[0] #Drzewo decyzyjne

print(wynik_dt, "MPa")

52.3 MPa


In [13]:
wynik_Xboost = model2.predict(predicted2)[0] #Gradient boost

print(wynik_Xboost, "MPa")

51.195663093324114 MPa


Wstawimy teraz zapisane dane do Excela

In [14]:
ws = xw.books.active.sheets[0]

wiek = predicted2.loc[0,"wiek [dni]"]

ws["B4"].value = [
    ["Przewidziana wytrzymałość na ściskanie po {:.0f} dniach na podstawie:".format(wiek)],
    ["Drzewa decyzjnego"], 
    ["Algorytmu wzmacniającego strukturę gradientu"]
    ]

ws["B4"].columns.autofit()

ws["C5"].value = [
    ["{:.2f} MPa".format(wynik_dt)],
    ["{:.2f} MPa".format(wynik_Xboost)]
    ]

## Jeżeli chcesz wykonać obliczenia dla innego zestawu danych to nie zamykaj pliku excel tylko wpisz ponowne wartości i uruchom ponownie poniższą komórkę

In [17]:
try:
    ws = xw.books.active.sheets[0]
except:
    xw.view(predicted)

own_predict = ws["A1:I2"].options(pd.DataFrame).value

wynik_dt = model.predict(own_predict)[0] #Drzewo decyzyjne
wynik_Xboost = model2.predict(own_predict)[0] #Gradient boost

wiek = own_predict.loc[0,"wiek [dni]"]

ws["B4"].value = [
    ["Przewidziana wytrzymałość na ściskanie po {:.0f} dniach na podstawie:".format(wiek)],
    ["Drzewa decyzjnego"], 
    ["Algorytmu wzmacniającego strukturę gradientu"]
    ]

ws["B4"].columns.autofit()

ws["C5"].value = [
    ["{:.2f} MPa".format(wynik_dt)],
    ["{:.2f} MPa".format(wynik_Xboost)]
    ]